# Test case for validate data with Monad

## import library

In [1]:
import pandas as pd
from data_lib import Monad_package as mp

## Prepare data that you want to validate and master data for checking

In [2]:
df = pd.read_parquet("dataset/trade_view_registration_case_for_prince.parquet")
# df["countryName"].unique()
df.head()

,countryName,city
0,INDIA,MAHARASTRA STAYTE
1,ARMENIA,d
2,BURKINA FASO,Banfora
3,PHILIPPINES,"Sta.Rosa, Laguna"
4,CHINA,Wuxi/jiangsu


In [3]:
master = pd.read_parquet("./dataset/master.parquet")
master.head()

,global_code,global_name,region_code,region_name,sub_region_code,sub_region_name,intermediate_region_code,intermediate_region_name,country_or_area,m49_code,iso_alpha2_code,iso_alpha3_code,year,id
0,001,World,142,Asia,034,Southern Asia,000,,Afghanistan,004,AF,AFG,2023,2023AFG
1,001,World,150,Europe,039,Southern Europe,000,,Albania,008,AL,ALB,2023,2023ALB
2,001,World,000,,000,,000,,Antarctica,010,AQ,ATA,2023,2023ATA
3,001,World,002,Africa,015,Northern Africa,000,,Algeria,012,DZ,DZA,2023,2023DZA
4,001,World,009,Oceania,061,Polynesia,000,,American Samoa,016,AS,ASM,2023,2023ASM


In [4]:
master_dict = master[["country_or_area","iso_alpha2_code"]].to_dict("records")
master_dict = {elm.get("country_or_area").lower():elm.get("iso_alpha2_code") for elm in master_dict}
print(f"The master data has : {len(master_dict)} records")

The master data has : 248 records


## Check column name 'countryName' in df dataframe

In [30]:
def country_name_01(x):
    valid_country = master_dict
    return valid_country[x.lower()]

def country_name_02(x):
    master_dict = {
        "laos" : "LA",
        "netherlands" : "NL",
        "turkey" : "TR",
        "iran" : "IR",
        "republic of south africa" : "ZA"
    }
    
    valid_country = master_dict
    return valid_country[x.lower()]

In [31]:
# "vietnam":"VN",
# "korea" : "KR",
# "taiwan": "TW",
# "united kingdom": "GB",
# "hong kong" : "HK",
# "russia" : "RU",

In [33]:
df_result = mp.clensing(df,"countryName",[country_name_01,country_name_02])
df_result

,countryName,city,countryName_result
0,INDIA,MAHARASTRA STAYTE,passed(IN)['Passed:country_name_01()']
1,ARMENIA,d,passed(AM)['Passed:country_name_01()']
2,BURKINA FASO,Banfora,passed(BF)['Passed:country_name_01()']
3,PHILIPPINES,"Sta.Rosa, Laguna",passed(PH)['Passed:country_name_01()']
4,CHINA,Wuxi/jiangsu,passed(CN)['Passed:country_name_01()']
...,...,...,...
59408,INDIA,Lucknow,passed(IN)['Passed:country_name_01()']
59409,INDIA,Shahjahanpur,passed(IN)['Passed:country_name_01()']
59410,THAILAND,Province,passed(TH)['Passed:country_name_01()']
59411,ISRAEL,,passed(IL)['Passed:country_name_01()']


In [34]:
df_result_explode = mp.explode_result(df_result,'countryName')
df_result_explode

,row,column,input,value,status,message
0,0,countryName,INDIA,IN,passed,Passed:country_name_01()
1,1,countryName,ARMENIA,AM,passed,Passed:country_name_01()
2,2,countryName,BURKINA FASO,BF,passed,Passed:country_name_01()
3,3,countryName,PHILIPPINES,PH,passed,Passed:country_name_01()
4,4,countryName,CHINA,CN,passed,Passed:country_name_01()
...,...,...,...,...,...,...
67247,59409,countryName,INDIA,IN,passed,Passed:country_name_01()
67248,59410,countryName,THAILAND,TH,passed,Passed:country_name_01()
67249,59411,countryName,ISRAEL,IL,passed,Passed:country_name_01()
67250,59412,countryName,NETHERLANDS,NL,passed,Failed:country_name_01():'netherlands'


In [35]:
df_fail = mp.get_dirty(df_result_explode)
df_fail

,row,column,input,value,status,message
11,10,countryName,UNITED KINGDOM,UNITED KINGDOM,dirty,Failed:country_name_01():'united kingdom'
12,10,countryName,UNITED KINGDOM,UNITED KINGDOM,dirty,Failed:country_name_02():'united kingdom'
14,12,countryName,KOREA,KOREA,dirty,Failed:country_name_01():'korea'
15,12,countryName,KOREA,KOREA,dirty,Failed:country_name_02():'korea'
22,19,countryName,HONG KONG,HONG KONG,dirty,Failed:country_name_01():'hong kong'
...,...,...,...,...,...,...
67162,59327,countryName,VIETNAM,VIETNAM,dirty,Failed:country_name_02():'vietnam'
67177,59341,countryName,UNITED KINGDOM,UNITED KINGDOM,dirty,Failed:country_name_01():'united kingdom'
67178,59341,countryName,UNITED KINGDOM,UNITED KINGDOM,dirty,Failed:country_name_02():'united kingdom'
67179,59342,countryName,UNITED KINGDOM,UNITED KINGDOM,dirty,Failed:country_name_01():'united kingdom'


In [36]:
test = df_fail.groupby("input").size().sort_values(ascending=False)
er_df = pd.DataFrame([test.index,test.values],["country","error_count"]).T
er_df.head()

,country,error_count
0,KOREA,2576
1,VIETNAM,2318
2,TAIWAN,1906
3,UNITED KINGDOM,1558
4,HONG KONG,1218


In [7]:
# place for experiments

## Note

In [8]:
# df_result = mo.check_countryName(df,'countryName')
# df_result

In [9]:
# df_result_explode = mo.explode_result(df_result,'countryName')
# df_result_explode

In [10]:
# df_fail = mo.get_dirty(df_result_explode)
# df_fail